## Baseline
This notebook implments a baseline model, which shows you how to handle the data and to provide a first very simple solution to the problem. You may re-use and modify any part of this notebook.

In [1]:
#!pip install torchmetrics --quiet

In [1]:
import os
import csv
import torch
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import re
import random
from tqdm.notebook import tqdm
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from torchmetrics import AUROC, F1Score
from nltk.corpus import stopwords

In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
# use weighted loss function due to the extreme imbalance of label
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset construction with Fasttext embedding.

from the code above, we can see that the comment has up to 315 words which is relatively lightweight. And due to the constraint of the time and resource, it is better to try lightweight embedding models. Given that the training set is user-generated-content which contains noise like typo it is recommended to use fasttext for its OOV performance. 

In [3]:
import fasttext
import fasttext.util
# fasttext.download_model('cc.en.300.bin')  # need to run this if no fasttext model in local.
ft = fasttext.load_model('/kaggle/input/cc-en-300-bin/cc.en.300.bin')

In [4]:
class FastTextDatasetWithDemographics(Dataset):
    def __init__(self, data_dir, mode, fasttext_model=None):
        super(FastTextDatasetWithDemographics, self).__init__()
        assert mode in ['train', 'val', 'test']
        self.mode = mode

        # Load the data
        self.data = pd.read_csv(os.path.join(data_dir, f'{mode}_x.csv'), index_col=0).fillna("na")

        # Load the labels if not the test set
        if self.mode != 'test':
            self.label = pd.read_csv(os.path.join(data_dir, f'{mode}_y.csv'))

        # Load FastText model
        if fasttext_model is None and self.mode == 'train':
            raise ValueError("FastText model must be provided for training.")
        self.fasttext_model = fasttext_model

        # Initialize stopwords
        self.stopwords = set(stopwords.words('english'))
    
    def clean_text(self, text):
        """Clean the text by removing special characters and stopwords."""
        # Remove special characters and lower the text
        text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
        # Remove stopwords
        text = ' '.join([word for word in text.split() if word not in self.stopwords])
        return text

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Extract the text comment
        x = self.data.iloc[idx, 0]
        # Replace the newline mark
        x = x.replace('\n', ' ').strip()
        # Clean the text by removing the stopwords.
        x = self.clean_text(x)

        # Compute FastText embedding
        x_embedding = self.fasttext_model.get_sentence_vector(x)
        x_tensor = torch.tensor(x_embedding).float()

        # If it's the test set, return just the embedding and index
        if self.mode == 'test':
            return x_tensor, idx
        else:
            # Otherwise, return the label and the demographics as well
            
            # the label
            y = torch.tensor([self.label["y"].iloc[idx]]).float()
            
            # process the demograhic features
            demographics = torch.tensor(self.label.iloc[idx,:8].values).float()
            
            return x_tensor, y, demographics, idx

In [5]:
train_dir = "/kaggle/input/toxic-comment-classification-dsba-2025/kaggle_data/"
val_dir = "/kaggle/input/toxic-comment-classification-dsba-2025/kaggle_data/"

In [6]:
train_dataset = FastTextDatasetWithDemographics(train_dir, 'train', ft)
val_dataset = FastTextDatasetWithDemographics(val_dir, 'val', train_dataset.fasttext_model)

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False)

# Model Training

We decided to separate streams of embedded text data and the demographic features for the sake of the interpretability of the contribution, flexibility, and a better representation learning.

In [8]:
class Attention(nn.Module):
    def __init__(self, embedding_dim):
        super(Attention, self).__init__()
        self.attention_weights = nn.Linear(embedding_dim,1)

    def forward(self, embeddings):
        weights = torch.softmax(self.attention_weights(embeddings), dim=1)
        attended_embeddings = embeddings * weights
        return attended_embeddings.sum(dim=1)

class ResidualBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(ResidualBlock, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.gelu = nn.GELU()
        self.fc2 = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        residual = x  # Shortcut connection
        out = self.fc1(x)
        out = self.gelu(out)
        out = self.fc2(out)
        return out + residual  # Add input back to output
        
class EnhancedToxicCommentClassifierWithAttention(nn.Module):
    def __init__(self, input_dim=300, hidden_dim=512, output_dim=1, num_residual_blocks=1):
        super(EnhancedToxicCommentClassifierWithAttention, self).__init__()
        self.attention = Attention(input_dim)
        self.fc_in = nn.Linear(input_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.gelu = nn.GELU()
        self.residual_blocks = nn.ModuleList(
            [ResidualBlock(hidden_dim, hidden_dim//2) for _ in range(num_residual_blocks)]
        )
        self.fc_out = nn.Linear(hidden_dim , output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text_embedding):
        text_embedding = text_embedding.unsqueeze(1)

        x = self.attention(text_embedding)
        x = self.fc_in(x)
        x = self.bn(x)
        x = self.gelu(x)
        for block in self.residual_blocks:
            x = block(x)
        x = self.fc_out(x)
        return self.sigmoid(x)

In [9]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# initiation of the model
model = EnhancedToxicCommentClassifierWithAttention()

# calculate the weight of each label. 
classes = pd.read_csv("/kaggle/input/toxic-comment-classification-dsba-2025/kaggle_data/train_y.csv")["y"]
class_counts = classes.value_counts().values.tolist()
total_samples = sum(class_counts)
weights = [total_samples / (len(class_counts) * count) for count in class_counts]
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([weights[1]])).to(device)

# optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.05, weight_decay=0.01)

# Use scheduler to better adjust the learning rate to facilidate learning.
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# different version

In [10]:
def compute_group_weights(group_accuracies):
    """
    Compute sample weights inversely proportional to group accuracy.
    If a sample belongs to multiple groups, its weight is averaged over those groups.
    """
    group_weights = 1.0 / (group_accuracies + 1e-6)  # Avoid division by zero
    group_weights /= group_weights.sum()  # Normalize so they sum to 1
    return torch.tensor(group_weights, dtype=torch.float32).to(device)

def group_accuracy(prediction, y):
    """
        Compute the worst group accuracy, with the groups being defined by ['male', 'female', 'LGBTQ',
        'christian', 'muslim', 'other_religions', 'black', 'white'] for positive and negative toxicity.
        arguments:
            prediction [pandas.DataFrame]: dataframe with 2 columns (index and pred)
            y [pandas.DataFrame]: dataframe containing the metadata
        returns:
            wga [float]: worst group accuracy
    """
    y.loc[prediction.index, 'pred'] = prediction.pred

    categories = ['male', 'female', 'LGBTQ', 'christian', 'muslim', 'other_religions', 'black', 'white']
    accuracies = []
    averaged_category_accuracies = []
    for category in categories:
        category_accuracies = []
        label_sizes = []
        for label in [0, 1]:
            group = y.loc[y[category] == label]
            label_sizes.append(len(group))
            group_accuracy = (group['y'] == (group['pred'] > 0.5)).mean()
            
            accuracies.append(group_accuracy)
            category_accuracies.append(group_accuracy)

        # calculate the averaged accuracy of this category using geometric average. 
        averaged_category_accuracy = np.exp(np.mean(np.log(np.array(category_accuracies) + 1e-6)))
        averaged_category_accuracies.append(averaged_category_accuracy)
        
    wga = np.min(accuracies)
    averaged_category_accuracies = torch.tensor(averaged_category_accuracies).float().to(device)
    return wga, averaged_category_accuracies

In [11]:
def train_model(model, optimizer, criterion, dataloader, group_accuracies, device): # paired with FasttextDataset with sample weight based on group performance
    """
        Train a model for one epoch.
        arguments:
            model [torch.nn.Module]: model to evaluate
            oprimizer [torch.optim]: optimizer used for training
            criterion [torch.nn.modules.loss]: desired loss to compute
            dataloader [torch.utils.data.DataLoader]: dataloader used for training
            device : cpu or gpu
        returns:
            dataset_loss [float]: computed loss on the dataset
            dataset_metric [float]: computed metric on the dataset
    """
    model.train()
    model.to(device)
    
    losses, predictions, indices = [], [], []
    for x, y, demographics, idx in tqdm(dataloader, leave=False):
        x, y, demographics= x.to(device), y.to(device), demographics.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred.squeeze(), y.squeeze().float())
        
        # compute the weighted loss using the sample weights which derived from the group_accuracies
        sample_weights = (demographics * compute_group_weights(group_accuracies)).sum(dim=1).to(device)
        weighted_loss = (loss * sample_weights).mean()  # Apply per-sample weight
        weighted_loss.backward()
        optimizer.step()

        losses.extend([weighted_loss.item()] * len(y))
        predictions.extend(pred.detach().squeeze().tolist())
        indices.extend(idx.tolist())
    
    pred_df = pd.DataFrame({'index': indices, 'pred': predictions})
    dataset_loss = np.mean(losses)
    wga, group_accuracies = group_accuracy(pred_df, y = dataloader.dataset.label)

    return dataset_loss, wga, group_accuracies

In [12]:
def evaluate_model(model, dataloader, criterion, group_accuracies, device): # # paired with FasttextDataset with sample weight based on group performance
    """
        Evaluate the model on a given dataloader.
        argument:
            model [torch.nn.Module]: model to evaluate
            dataloader [torch.utils.data.DataLoader]: dataloader on which to evaluate
            criterion [torch.nn.modules.loss]: desired loss to compute
            device : cpu or gpu
        returns:
            dataset_loss [float]: computed loss on the dataset
            dataset_metric [float]: computed metric on the dataset
    """
    model.eval()
    model.to(device)
    losses, predictions, indices = [], [], []
    for x, y, demographics, idx in tqdm(dataloader, leave=False):
        x, y, demographics= x.to(device), y.to(device), demographics.to(device)
        with torch.no_grad():
            pred = model(x)
        sample_weights = (demographics * compute_group_weights(group_accuracies)).sum(dim=1).to(device)    
        loss = criterion(pred.squeeze(), y.squeeze().float())

        # compute the weighted loss using the sample weights which derived from the group_accuracies
        weighted_loss = (loss * sample_weights).mean() 
        losses.extend([weighted_loss.item()] * len(y))
        predictions.extend(pred.detach().squeeze().tolist())
        indices.extend(idx.tolist())


    pred_df = pd.DataFrame({'index': indices, 'pred': predictions})
    dataset_loss = np.mean(losses)
    wga, group_accuracies = group_accuracy(pred_df, dataloader.dataset.label)
    return dataset_loss, wga, group_accuracies

In [16]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

model = EnhancedToxicCommentClassifierWithAttention()

classes = pd.read_csv("/kaggle/input/toxic-comment-classification-dsba-2025/kaggle_data/train_y.csv")["y"]
class_counts = classes.value_counts().values.tolist()
total_samples = sum(class_counts)
weights = [total_samples / (len(class_counts) * count) for count in class_counts]
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([weights[1]])).to(device)

optimizer = optim.AdamW(model.parameters(), lr=0.05, weight_decay=0.01)
# Use scheduler to better adjust the learning rate to facilidate learning.
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

In [17]:
num_epochs = 10
group_accuracies = torch.full((8,), 0.5, dtype=torch.float32)  # Default accuracy = 50%
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss, wga, _ = train_model(model,optimizer, criterion, train_dataloader, group_accuracies, device)
    print(f"Epoch {epoch + 1} - train Loss: {train_loss:.4f}, WGA: {wga:.4f}")
    mlp_val_loss, val_wga, group_accuracies = evaluate_model(model, val_dataloader, criterion, group_accuracies, device)
    #scheduler.step(mlp_val_loss)
    print(group_accuracies)
    print(f"Epoch {epoch + 1} - evaluation loss {mlp_val_loss:.4f}, WGA: {val_wga:.4f}")

Epoch 1/10


  0%|          | 0/2102 [00:00<?, ?it/s]

<ipython-input-10-236d4c1d977e>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(group_weights, dtype=torch.float32).to(device)


Epoch 1 - train Loss: 0.0664, WGA: 0.6880


  0%|          | 0/353 [00:00<?, ?it/s]

...

tensor([0.8710, 0.8800, 0.8205, 0.9002, 0.8231, 0.8621, 0.7791, 0.7956])
Epoch 9 - evaluation loss 0.0718, WGA: 0.6774
Epoch 10/10


  0%|          | 0/2102 [00:00<?, ?it/s]

Epoch 10 - train Loss: 0.0650, WGA: 0.6701


  0%|          | 0/353 [00:00<?, ?it/s]

tensor([0.8572, 0.8652, 0.8140, 0.8865, 0.8067, 0.8369, 0.7805, 0.7922])
Epoch 10 - evaluation loss 0.0707, WGA: 0.6937


# Generate Test Result

Once we are happy with our results, we want to make a prediction on the test set. Your submission `.csv` file should contain 2 columns:
- ID: with the id of each prediction (do not shuffle to not mix things up)
- pred: the prediction of the model (thresholded or not)

In [21]:
#model = RandomClassifier()
#test_dataset = FastTextDataset(train_dir, 'test', train_dataset.fasttext_model)
test_dataset = FastTextDatasetWithDemographics(train_dir, 'test', train_dataset.fasttext_model)
test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=False)
model.eval()
test_predictions, indices = [], []
for x, idx in tqdm(test_dataloader, leave=False):
    x = x.to(device)
    with torch.no_grad():
        pred = (model(x).squeeze() > 0.5).int()
    test_predictions.extend(pred.tolist())
    indices.extend(idx.tolist())

  0%|          | 0/262 [00:00<?, ?it/s]

In [22]:
pred_df = pd.DataFrame({'ID': indices, 'pred': test_predictions})
pred_df.to_csv('prediction.csv', index=False)